In [1]:
# 导入各种包
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate
import warnings; warnings.simplefilter('ignore')

In [2]:
# 读取数据
md = pd. read_csv('/Users/apple/Desktop/5003-project/moviedata/TMD/movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
# 把genres这一栏的数据重新整理：
# 先给空白数据填充[],然后用literal_eval这个字符串变为原有的类型；
# 这一行的目的是提取电影题材的关键词；
# 下面代码的虚函数里，x是某一行的list，而i指的是list中的字典；
# 这里的apply函数是pandas中的apply函数，可以将数据帧中的许多列遍历计算，和map的区别在于参数不同，这个全都是函数式编程的形式在进行；
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
# 每个电影被投了多少票
md['vote_count']
print(type(md))
print(type(md['vote_count']))
print(type(md['vote_count'].notnull()))
print(md.shape)
print(md['vote_count'].shape)
print(md['vote_count'].notnull().shape)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
(45466, 24)
(45466,)
(45466,)


In [5]:
md['vote_count'].notnull().head()

0    True
1    True
2    True
3    True
4    True
Name: vote_count, dtype: bool

In [6]:
md[md['vote_count'].notnull()]['vote_count'].head()

0    5415.0
1    2413.0
2      92.0
3      34.0
4     173.0
Name: vote_count, dtype: float64

In [7]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
print(vote_averages)

0        7
1        6
2        6
3        6
4        5
5        7
6        6
7        5
8        5
9        6
10       6
11       5
12       7
13       7
14       5
15       7
16       7
17       6
18       6
19       5
20       6
21       6
22       6
23       6
24       7
25       7
26       6
27       7
28       7
29       6
        ..
45436    3
45437    8
45438    6
45439    5
45440    5
45441    5
45442    5
45443    7
45444    6
45445    4
45446    6
45447    6
45448    5
45449    5
45450    6
45451    6
45452    0
45453    6
45454    4
45455    6
45456    6
45457    7
45458    7
45459    3
45460    5
45461    4
45462    9
45463    3
45464    0
45465    0
Name: vote_average, Length: 45460, dtype: int64


In [8]:
C = vote_averages.mean()
C
print(C)

5.244896612406511


In [9]:
type(md[md['vote_count'].notnull()]['vote_count'])

pandas.core.series.Series

In [10]:
# 这里不是说有多少个，而是0.95的分位点的counts的值是多少；
m = vote_counts.quantile(0.95)
m

434.0

In [11]:
# 转换时间的格式，只提取年份；
# 参数解释：errors：If ‘coerce’, then invalid parsing will be set as NaT
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [12]:
# 先看看这个返回值，是一个45466行的布尔值；
md['vote_count'] >= m
print((md['vote_count'] >= m).shape)
type(md['vote_count'] >= m)

(45466,)


pandas.core.series.Series

In [13]:
type((md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull()))

pandas.core.series.Series

In [14]:
temmmp = (md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())
temmmp.shape

(45466,)

In [15]:
# 这里用&运算符得出很骚的结果
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
print(qualified)
# 将票数的类型转换一下，改为整型
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

                                             title  year  vote_count  \
0                                        Toy Story  1995      5415.0   
1                                          Jumanji  1995      2413.0   
5                                             Heat  1995      1886.0   
9                                        GoldenEye  1995      1194.0   
15                                          Casino  1995      1343.0   
17                                      Four Rooms  1995       539.0   
18                  Ace Ventura: When Nature Calls  1995      1128.0   
31                                  Twelve Monkeys  1995      2470.0   
33                                            Babe  1995       756.0   
38                                        Clueless  1995       828.0   
43                                   Mortal Kombat  1995       452.0   
46                                           Se7en  1995      5915.0   
47                                      Pocahontas  1995      15

(2274, 6)

# 完成了数据的转换，下面开始推荐系统的构建
* v is the number of votes for the movie
* m is the minimum votes required to be listed in the chart
* R is the average rating of the movie
* C is the mean vote across the whole report

In [16]:
# V是这个电影收获的投票的数量
# R是这个电影收获的平均评分
# m是临界值，95%的电影都至少收获434票，那就要多于这个值才参与讨论
# C是TMBD上所有电影的平均评分
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [17]:
# axis : {0 or ‘index’, 1 or ‘columns’}, default 0
# 0 or ‘index’: apply function to each column
# 1 or ‘columns’: apply function to each row
# 加的这一栏是加权评分，不知道图啥
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [18]:
# 看一下数据的大体构成
qualified.sort_values('wr', ascending=False).head(50)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


In [19]:
# s是把电影的类型分别提取出来，前面的数字是电影的序号。
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
print(s)

0              Animation
0                 Comedy
0                 Family
1              Adventure
1                Fantasy
1                 Family
2                Romance
2                 Comedy
3                 Comedy
3                  Drama
3                Romance
4                 Comedy
5                 Action
5                  Crime
5                  Drama
5               Thriller
6                 Comedy
6                Romance
7                 Action
7              Adventure
7                  Drama
7                 Family
8                 Action
8              Adventure
8               Thriller
9              Adventure
9                 Action
9               Thriller
10                Comedy
10                 Drama
              ...       
45446             Comedy
45449          Animation
45449             Family
45450            Fantasy
45450             Action
45450           Thriller
45451             Comedy
45451            Fantasy
45452        Documentary


In [20]:
s.name = 'genre'
# gen_md大体上是把s和之前的md（先删掉没被处理过的‘genre’部分）连接起来；
gen_md = md.drop('genres', axis=1).join(s)
print(gen_md)

       adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
1      False                                                NaN  65000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
3      False                                                NaN  16000000   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

In [21]:
# 这里构建一个需要获得85%的电影都能获得的最低票数的电影的表；
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250) 
    return qualified

In [22]:
build_chart('Comedy').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.463024
351,Forrest Gump,1994,8147,8,48.3072,7.963363
1225,Back to the Future,1985,6239,8,25.7785,7.952358
18465,The Intouchables,2011,5410,8,16.0869,7.945207
22841,The Grand Budapest Hotel,2014,4644,8,14.442,7.936384
2211,Life Is Beautiful,1997,3643,8,39.395,7.919430
732,Dr. Strangelove or: How I Learned to Stop Worr...,1964,1472,8,9.80398,7.809073
3342,Modern Times,1936,881,8,8.15956,7.695554
883,Some Like It Hot,1959,835,8,11.8451,7.680781
1236,The Great Dictator,1940,756,8,9.24175,7.651762


* 上面的推荐系统是不包含任何用户信息的，它只能对某一个电影类型做出推荐，我们希望给每个用户提供独有的推荐信息，这就需要我们构建一个基于内容的过滤器。

In [23]:
# 读取数据
links_small = pd.read_csv('/Users/apple/Desktop/5003-project/moviedata/TMD/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [24]:
md.shape

(45466, 25)

In [25]:
md

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995
5,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,1995
6,False,NaN,58000000,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0,1995
7,False,NaN,0,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0,1995
8,False,NaN,35000000,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0,1995
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,1995


In [26]:
md.iloc[[19730]]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[Carousel Productions, Vision View Entertainme...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [27]:
md.iloc[[29503]]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[Aniplex, GoHands, BROSTA TV, Mardock Scramble...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [28]:
md.iloc[[35587]]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[Odyssey Media, Pulser Productions, Rogue Stat...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [29]:
# drop了这三行的数据，不知道为啥这么针对
md = md.drop([19730, 29503, 35587])

In [30]:
md.shape

(45463, 25)

In [31]:
md

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995
5,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,1995
6,False,NaN,58000000,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0,1995
7,False,NaN,0,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0,1995
8,False,NaN,35000000,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0,1995
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,1995


In [32]:
# 把id这一栏的类型转换一下
md['id'] = md['id'].astype('int')

In [33]:
# isin函数，Return boolean DataFrame showing whether each element in the DataFrame is contained in values.
# 返回是布尔值，来表示前面的部分中的每个元素是否包含你传递进来的参数
# 所以这里smd得到的是md中在links_small中出现过的部分
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

In [34]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [35]:
smd['tagline'].head(10)

0                                                     
1            Roll the dice and unleash the excitement!
2    Still Yelling. Still Fighting. Still Ready for...
3    Friends are the people who let you be yourself...
4    Just When His World Is Back To Normal... He's ...
5                             A Los Angeles Crime Saga
6    You are cordially invited to the most surprisi...
7                               The Original Bad Boys.
8                           Terror goes into overtime.
9                 No limits. No fears. No substitutes.
Name: tagline, dtype: object

In [36]:
smd['overview'].head(10)

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
5    Obsessive master thief, Neil McCauley leads a ...
6    An ugly duckling having undergone a remarkable...
7    A mischievous young boy, Tom Sawyer, witnesses...
8    International action superstar Jean Claude Van...
9    James Bond must unmask the mysterious head of ...
Name: overview, dtype: object

In [37]:
smd['tagline'].head(10)

0                                                     
1            Roll the dice and unleash the excitement!
2    Still Yelling. Still Fighting. Still Ready for...
3    Friends are the people who let you be yourself...
4    Just When His World Is Back To Normal... He's ...
5                             A Los Angeles Crime Saga
6    You are cordially invited to the most surprisi...
7                               The Original Bad Boys.
8                           Terror goes into overtime.
9                 No limits. No fears. No substitutes.
Name: tagline, dtype: object

In [38]:
smd['description'].head(10)

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
5    Obsessive master thief, Neil McCauley leads a ...
6    An ugly duckling having undergone a remarkable...
7    A mischievous young boy, Tom Sawyer, witnesses...
8    International action superstar Jean Claude Van...
9    James Bond must unmask the mysterious head of ...
Name: description, dtype: object

In [39]:
# 这里应该在做一个语义分析，TfidfVectorizer是tflearn中的包。
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [40]:
tfidf_matrix.shape

(9099, 268124)

In [41]:
tfidf_matrix

<9099x268124 sparse matrix of type '<class 'numpy.float64'>'
	with 540591 stored elements in Compressed Sparse Row format>

In [42]:
# 这里调用sklearn's linear_kernel来进行计算余弦相似度
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(9099, 9099)

In [43]:
# 现在cosine_s im是一个9099*9099的矩阵，保存了9099个数据两两之间的xiangsid
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [44]:
type(smd)

pandas.core.frame.DataFrame

In [45]:
# 重新设置索引，并不会drop之前的而是新建
smd = smd.reset_index()
titles = smd['title']
# 这里定义了indices，这里把title作为索引！
indices = pd.Series(smd.index, index=smd['title'])

In [46]:
smd.shape

(9099, 27)

In [47]:
print(smd['title'])
print(smd['title'].shape)
indices

0                                               Toy Story
1                                                 Jumanji
2                                        Grumpier Old Men
3                                       Waiting to Exhale
4                             Father of the Bride Part II
5                                                    Heat
6                                                 Sabrina
7                                            Tom and Huck
8                                            Sudden Death
9                                               GoldenEye
10                                 The American President
11                            Dracula: Dead and Loving It
12                                                  Balto
13                                                  Nixon
14                                       Cutthroat Island
15                                                 Casino
16                                  Sense and Sensibility
17            

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
Heat                                                     5
Sabrina                                                  6
Tom and Huck                                             7
Sudden Death                                             8
GoldenEye                                                9
The American President                                  10
Dracula: Dead and Loving It                             11
Balto                                                   12
Nixon                                                   13
Cutthroat Island                                        14
Casino                                                  15
Sense and Sensibility                             

In [48]:
indices.shape

(9099,)

In [49]:
# 所以indices在这里应该是全局的，不是传进去的
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [50]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [51]:
get_recommendations('Inception').head(10)

5239                              Cypher
141                                Crumb
6398                         Renaissance
653                            Lone Star
1703                               House
4739                    The Pink Panther
319                                 Cobb
2828    What Ever Happened to Baby Jane?
8867                     Pitch Perfect 2
979          Once Upon a Time in America
Name: title, dtype: object

In [52]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

In [53]:
temd=md

In [54]:
md=temd

* 需要进一步优化

In [55]:
credits = pd.read_csv('/Users/apple/Desktop/5003-project/moviedata/TMD/credits.csv')
keywords = pd.read_csv('/Users/apple/Desktop/5003-project/moviedata/TMD/keywords.csv')

In [56]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


In [57]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [58]:
md.shape

(45463, 25)

In [59]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')
print(md.shape)
md

(46628, 28)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
5,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,1995,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
6,False,NaN,58000000,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0,1995,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...","[{'id': 90, 'name': 'paris'}, {'id': 380, 'nam..."
7,False,NaN,0,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0,1995,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",[]
8,False,NaN,35000000,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0,1995,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...","[{'id': 949, 'name': 'terrorist'}, {'id': 1562..."
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,1995,"[{'cast_id': 1, 'character': 'James Bond',

In [108]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [109]:
# 演员表
smd['cast'].head(10)

0    [{'cast_id': 14, 'character': 'Woody (voice)',...
1    [{'cast_id': 1, 'character': 'Alan Parrish', '...
2    [{'cast_id': 2, 'character': 'Max Goldman', 'c...
3    [{'cast_id': 1, 'character': "Savannah 'Vannah...
4    [{'cast_id': 1, 'character': 'George Banks', '...
5    [{'cast_id': 25, 'character': 'Lt. Vincent Han...
6    [{'cast_id': 1, 'character': 'Linus Larrabee',...
7    [{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...
8    [{'cast_id': 1, 'character': 'Darren Francis T...
9    [{'cast_id': 1, 'character': 'James Bond', 'cr...
Name: cast, dtype: object

In [110]:
# 职员表，这个部分就取导演
smd['crew'].head(10)

0    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2    [{'credit_id': '52fe466a9251416c75077a89', 'de...
3    [{'credit_id': '52fe44779251416c91011acb', 'de...
4    [{'credit_id': '52fe44959251416c75039ed7', 'de...
5    [{'credit_id': '52fe4292c3a36847f802916d', 'de...
6    [{'credit_id': '52fe44959251416c75039da9', 'de...
7    [{'credit_id': '52fe46bdc3a36847f810f797', 'de...
8    [{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...
9    [{'credit_id': '52fe426ec3a36847f801e14b', 'de...
Name: crew, dtype: object

In [111]:
# 这个数据帧中的每一行，都是一个str，因此在下面要用literal_eval函数转换成list
type(smd['crew'][0])

str

In [112]:
# literal_eval函数了解一下，比eval安全
type(literal_eval(smd['crew'][0]))

list

In [113]:
# 数据类型大转换
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

* 这部分的数据处理差不多了

In [114]:
# 这个函数，是从crew这一栏中取出导演的数据来
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
# 如果这一栏没有导演，那就返回空咯
    return np.nan

In [115]:
smd['director'] = smd['crew'].apply(get_director)

In [116]:
smd['director']

0                  John Lasseter
1                   Joe Johnston
2                  Howard Deutch
3                Forest Whitaker
4                  Charles Shyer
5                   Michael Mann
6                 Sydney Pollack
7                   Peter Hewitt
8                    Peter Hyams
9                Martin Campbell
10                    Rob Reiner
11                    Mel Brooks
12                   Simon Wells
13                  Oliver Stone
14                  Renny Harlin
15               Martin Scorsese
16                       Ang Lee
17                Allison Anders
18                Steve Oedekerk
19                  Joseph Ruben
20              Barry Sonnenfeld
21                     Jon Amiel
22                Richard Donner
23                  Victor Salva
24                   Mike Figgis
25                 Oliver Parker
26           Lesli Linka Glatter
27                 Roger Michell
28            Jean-Pierre Jeunet
29                   Zhang Yimou
          

In [117]:
type(smd)

pandas.core.frame.DataFrame

In [118]:
# 先把输入的cast写好，再取前三，想想能不能合在一起；
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [119]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [120]:
# bytes.replace(old, new[, count])
# 将更大的空格改成小一点的
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [121]:
# 把导演的权重变为原来的三倍，参照这个方式修改其他的权重
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x,x])

In [122]:
# 计算一下每个关键词出现的次数
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [123]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [124]:
s.count()

12940

In [125]:
# 只出现一次的关键词在推荐的时候没有任何意义，不过这里导演已经被乘以三了，改写时候注意有没有影响到
s = s[s > 1]
s.count()

6709

In [126]:
# 让意思相近的单词视为同一个意思，比如这里的dogs和dog
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [127]:
stemmer.stem('tiger')

'tiger'

In [128]:
# s是关键词，这里过滤所有传递进来的词组在s中出现的，将其存进words中，然后返回words
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [129]:
smd['keywords'].shape

(9219,)

In [130]:
# 调用过滤函数
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'].shape

(9219,)

In [131]:
# 将意思相近的单词过滤
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'].shape

(9219,)

In [132]:
# 转换大小写，并修正空格的问题
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

* 这里的每一行都有很多的关键词，所以过滤之后的行数并没有减少，并且用shape是查看不了的，因为其实不同的关键词被合称了string存储在一起。

In [137]:
smd['keywords']

0        [jealousi, toy, boy, friendship, friend, rival...
1        [boardgam, disappear, basedonchildren'sbook, n...
2                   [fish, bestfriend, duringcreditssting]
3        [basedonnovel, interracialrelationship, single...
4        [babi, midlifecrisi, confid, age, daughter, mo...
5        [robberi, detect, bank, obsess, chase, shoot, ...
6        [pari, brotherbrotherrelationship, chauffeur, ...
7                                                       []
8                  [terrorist, hostag, explos, vicepresid]
9        [cuba, falselyaccus, secretident, computerviru...
10                   [whitehous, usapresid, newlov, widow]
11                                        [dracula, spoof]
12                   [wolf, alaska, dog, goos, bearattack]
13       [usapresid, presidentialelect, watergatescand,...
14        [exoticisland, treasur, map, ship, scalp, pirat]
15       [poker, drugabus, 1970s, overdos, illegalprost...
16       [bowl, basedonnovel, servant, countrylif, jane.

In [138]:
smd['cast']

0                         [tomhanks, timallen, donrickles]
1              [robinwilliams, jonathanhyde, kirstendunst]
2                 [waltermatthau, jacklemmon, ann-margret]
3           [whitneyhouston, angelabassett, lorettadevine]
4                  [stevemartin, dianekeaton, martinshort]
5                      [alpacino, robertdeniro, valkilmer]
6                 [harrisonford, juliaormond, gregkinnear]
7        [jonathantaylorthomas, bradrenfro, rachaelleig...
8        [jean-claudevandamme, powersboothe, dorianhare...
9              [piercebrosnan, seanbean, izabellascorupco]
10           [michaeldouglas, annettebening, michaelj.fox]
11                  [leslienielsen, melbrooks, amyyasbeck]
12                  [kevinbacon, bobhoskins, bridgetfonda]
13               [anthonyhopkins, joanallen, powersboothe]
14              [geenadavis, matthewmodine, franklangella]
15                   [robertdeniro, sharonstone, joepesci]
16                  [katewinslet, emmathompson, hughgran

In [139]:
smd['director']

0               [johnlasseter, johnlasseter, johnlasseter]
1                  [joejohnston, joejohnston, joejohnston]
2               [howarddeutch, howarddeutch, howarddeutch]
3         [forestwhitaker, forestwhitaker, forestwhitaker]
4               [charlesshyer, charlesshyer, charlesshyer]
5                  [michaelmann, michaelmann, michaelmann]
6            [sydneypollack, sydneypollack, sydneypollack]
7                  [peterhewitt, peterhewitt, peterhewitt]
8                     [peterhyams, peterhyams, peterhyams]
9         [martincampbell, martincampbell, martincampbell]
10                       [robreiner, robreiner, robreiner]
11                       [melbrooks, melbrooks, melbrooks]
12                    [simonwells, simonwells, simonwells]
13                 [oliverstone, oliverstone, oliverstone]
14                 [rennyharlin, rennyharlin, rennyharlin]
15        [martinscorsese, martinscorsese, martinscorsese]
16                                [anglee, anglee, angle

In [140]:
smd['genres']

0                              [Animation, Comedy, Family]
1                             [Adventure, Fantasy, Family]
2                                        [Romance, Comedy]
3                                 [Comedy, Drama, Romance]
4                                                 [Comedy]
5                         [Action, Crime, Drama, Thriller]
6                                        [Comedy, Romance]
7                       [Action, Adventure, Drama, Family]
8                            [Action, Adventure, Thriller]
9                            [Adventure, Action, Thriller]
10                                [Comedy, Drama, Romance]
11                                        [Comedy, Horror]
12                          [Family, Animation, Adventure]
13                                        [History, Drama]
14                                     [Action, Adventure]
15                                          [Drama, Crime]
16                                        [Drama, Romanc

In [157]:
# 把所有的加在一起，称为soup
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
print(smd['soup'])
print(type(smd['soup']))

0        [jealousi, toy, boy, friendship, friend, rival...
1        [boardgam, disappear, basedonchildren'sbook, n...
2        [fish, bestfriend, duringcreditssting, walterm...
3        [basedonnovel, interracialrelationship, single...
4        [babi, midlifecrisi, confid, age, daughter, mo...
5        [robberi, detect, bank, obsess, chase, shoot, ...
6        [pari, brotherbrotherrelationship, chauffeur, ...
7        [jonathantaylorthomas, bradrenfro, rachaelleig...
8        [terrorist, hostag, explos, vicepresid, jean-c...
9        [cuba, falselyaccus, secretident, computerviru...
10       [whitehous, usapresid, newlov, widow, michaeld...
11       [dracula, spoof, leslienielsen, melbrooks, amy...
12       [wolf, alaska, dog, goos, bearattack, kevinbac...
13       [usapresid, presidentialelect, watergatescand,...
14       [exoticisland, treasur, map, ship, scalp, pira...
15       [poker, drugabus, 1970s, overdos, illegalprost...
16       [bowl, basedonnovel, servant, countrylif, jane.

In [160]:
help(' '.join)

Help on built-in function join:

join(...) method of builtins.str instance
    S.join(iterable) -> str
    
    Return a string which is the concatenation of the strings in the
    iterable.  The separator between elements is S.



In [158]:
# S.join函数，把S作为分隔符，join所有iterable对象称为一个str
# 这里就是，通过空格把可迭代对象连接成字符串
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))
print(smd['soup'])
print(type(smd['soup']))

0        jealousi toy boy friendship friend rivalri boy...
1        boardgam disappear basedonchildren'sbook newho...
2        fish bestfriend duringcreditssting waltermatth...
3        basedonnovel interracialrelationship singlemot...
4        babi midlifecrisi confid age daughter motherda...
5        robberi detect bank obsess chase shoot thief h...
6        pari brotherbrotherrelationship chauffeur long...
7        jonathantaylorthomas bradrenfro rachaelleighco...
8        terrorist hostag explos vicepresid jean-claude...
9        cuba falselyaccus secretident computervirus se...
10       whitehous usapresid newlov widow michaeldougla...
11       dracula spoof leslienielsen melbrooks amyyasbe...
12       wolf alaska dog goos bearattack kevinbacon bob...
13       usapresid presidentialelect watergatescand bio...
14       exoticisland treasur map ship scalp pirat geen...
15       poker drugabus 1970s overdos illegalprostitut ...
16       bowl basedonnovel servant countrylif janeauste.

* [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [164]:
# 这个方法的解释在上方，fit_transform生成一个矩阵
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])
count.get_feature_names()

['11',
 '11 jessicachastain',
 '11 thomashorn',
 '15thcenturi',
 '15thcenturi ottomanempir',
 '15thcenturi tomhulce',
 '16thcenturi',
 '16thcenturi alpacino',
 '16thcenturi catherinemccormack',
 '16thcenturi conquest',
 '16thcenturi drewbarrymore',
 '16thcenturi toshirōmifune',
 '17thcenturi',
 '17thcenturi 3d',
 '17thcenturi anyataylor',
 '17thcenturi douglasfairbanks',
 '17thcenturi song',
 '17thcenturi tatsuyanakadai',
 '18thcenturi',
 '18thcenturi charlesberling',
 '18thcenturi danielday',
 '18thcenturi ghost',
 '18thcenturi indigenouspeopl',
 '18thcenturi learn',
 '18thcenturi liamneeson',
 '18thcenturi murrayabraham',
 '18thcenturi oppositesattract',
 '1910s',
 '1910s careymulligan',
 '1910s christopherreeve',
 '1910s gerardbutler',
 '1910s katewinslet',
 '1910s stuartwhitman',
 '1920s',
 '1920s 1930s',
 '1920s brendanfraser',
 '1920s catincauntaru',
 '1920s janemarch',
 '1920s jenniferjasonleigh',
 '1920s judydavis',
 '1930s',
 '1930s 20thcenturi',
 '1930s billmurray',
 '1930s c

In [166]:
count_matrix.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [169]:
# 两两组合后的关键词种类还挺多的
count_matrix.toarray().shape

(9219, 107377)

In [170]:
# 看看每一行都有多少个关键词出现了
for i in count_matrix.toarray():
    print(sum(i))

35
31
23
27
31
69
25
19
27
47
27
19
27
27
27
25
35
37
25
27
51
29
57
41
47
19
29
21
47
31
35
37
35
27
63
19
21
31
17
25
25
27
39
29
35
43
55
21
31
33
17
31
23
27
27
19
27
23
35
35
23
19
27
27
21
65
97
21
27
19
23
13
25
29
21
57
63
25
11
27
41
25
21
27
29
21
19
55
21
53
29
19
35
39
15
39
23
49
29
13
35
37
37
15
19
31
33
27
23
19
29
41
15
31
17
15
19
31
27
25
19
21
33
27
53
25
25
23
21
41
25
27
55
41
19
27
15
39
25
33
41
19
53
31
61
33
15
27
25
49
27
35
39
23
19
21
31
15
29
33
29
23
21
33
23
25
21
25
37
27
27
27
21
47
69
25
13
25
21
29
25
19
15
29
27
15
13
17
33
37
15
27
35
17
23
33
21
27
29
27
21
29
51
49
29
25
37
59
33
27
23
33
17
33
25
17
15
19
35
45
15
27
21
15
31
27
25
21
29
35
29
23
49
25
51
21
15
37
29
31
21
35
23
55
27
27
17
25
31
33
15
29
33
19
41
23
27
17
39
29
71
35
37
49
25
21
45
39
31
27
23
25
29
35
33
25
27
11
17
15
23
21
39
31
33
29
23
31
17
31
13
27
31
39
27
39
15
31
13
17
21
27
27
37
33
19
39
45
29
21
31
25
35
27
67
39
33
31
23
19
41
47
53
25
23
25
33
27
31
25
63
33
39
3

19
49
17
25
27
25
13
31
31
27
53
41
35
21
35
37
31
29
25
27
23
13
21
51
15
29
39
25
19
19
21
45
27
19
35
35
13
21
25
29
43
37
35
49
23
17
19
23
25
35
21
35
25
33
25
37
45
17
27
17
39
25
25
29
27
21
19
27
39
39
25
19
33
33
15
61
21
29
37
15
25
23
43
19
27
23
19
25
21
35
29
41
13
25
29
21
17
47
23
17
25
27
33
41
19
33
35
27
41
21
27
41
27
25
35
19
29
31
33
39
21
19
15
55
29
23
19
21
17
45
15
47
19
35
37
39
19
25
13
21
39
35
15
27
27
25
15
19
27
21
21
7
19
23
19
27
35
27
55
37
15
15
55
15
59
35
21
27
19
41
29
65
43
69
19
45
53
23
25
15
25
23
31
25
25
27
27
33
17
27
13
17
19
17
39
15
45
29
25
13
27
41
19
49
25
29
35
31
37
23
27
31
35
57
29
21
35
25
43
31
37
71
29
29
31
29
27
51
73
29
35
47
29
33
27
61
23
17
17
23
23
57
19
17
21
19
27
29
35
45
53
27
31
27
49
55
29
25
23
35
29
43
31
27
25
37
37
45
37
31
29
23
23
47
33
21
27
49
17
31
47
53
21
25
25
21
27
35
23
21
13
19
23
17
35
33
23
27
41
51
17
19
29
15
29
21
17
31
35
21
37
25
31
19
21
27
23
23
31
27
15
19
33
23
21
33
31
13
39
25
29
25
21
25

29
27
23
45
39
27
29
21
33
25
45
49
39
25
37
41
23
23
21
27
21
33
17
17
23
19
27
21
29
19
31
15
27
21
23
31
27
37
25
19
37
23
23
15
19
39
19
29
7
15
39
41
35
23
37
27
37
43
43
21
25
29
23
39
57
35
29
35
15
29
31
17
31
19
13
25
17
17
27
23
15
15
27
45
21
23
39
29
21
31
29
9
21
25
21
45
23
29
21
45
29
23
31
33
49
19
27
21
21
21
21
21
21
21
21
29
15
43
19
31
45
27
39
33
29
23
27
27
29
23
9
21
43
19
25
27
19
27
19
19
19
21
29
15
15
21
23
31
37
35
19
33
23
47
15
35
33
37
29
21
27
25
13
21
37
21
47
35
19
49
37
37
17
23
61
25
25
21
47
19
19
17
25
13
15
19
49
41
17
17
33
25
31
33
39
11
21
29
49
41
33
19
37
43
25
29
19
29
51
27
41
13
49
15
29
21
61
21
7
15
23
19
19
19
21
25
37
23
25
29
21
27
43
19
39
37
31
37
41
41
25
43
23
47
21
15
15
17
35
41
35
37
29
33
21
33
35
21
31
23
31
27
17
17
27
19
37
17
21
37
41
23
21
33
23
21
23
41
29
23
19
33
23
27
15
21
19
23
23
19
39
19
19
27
21
47
35
25
29
21
37
21
29
13
19
17
23
41
29
33
19
39
25
23
35
15
45
35
33
37
21
21
25
7
25
15
17
25
37
23
17
17
27
41
55


7
11
15
15
15
37
21
53
13
43
47
37
29
35
23
17
19
39
43
17
25
19
39
19
23
19
29
17
43
19
21
19
31
29
21
21
21
21
21
21
21
21
21
15
19
35
17
13
9
19
51
33
25
15
13
17
73
29
27
31
21
23
25
23
25
53
21
29
45
23
17
29
21
51
17
13
35
21
31
21
25
17
17
15
23
15
19
15
13
29
11
15
17
33
13
25
13
19
23
11
23
47
15
19
23
31
21
21
27
15
31
29
25
23
9
19
15
39
15
15
21
29
53
25
19
33
41
17
37
37
21
29
27
27
17
37
19
27
17
31
23
19
23
21
33
29
47
45
27
23
15
25
27
15
41
25
37
39
25
17
21
27
17
27
23
27
13
23
39
21
45
17
41
15
39
29
15
35
29
35
15
33
21
35
27
13
29
31
25
27
23
37
41
45
15
27
49
51
23
31
35
13
33
13
33
39
23
35
21
21
21
31
27
43
13
27
25
25
11
21
27
13
33
29
23
19
31
27
33
33
21
23
25
31
25
33
61
13
17
17
17
17
15
29
23
29
41
21
27
23
35
63
17
31
41
35
15
15
27
49
25
23
43
27
33
19
47
25
49
33
27
39
25
19
23
15
37
35
17
17
25
15
23
35
13
29
37
21
21
21
21
69
29
37
17
21
43
23
27
41
27
47
21
21
29
39
33
23
23
29
17
23
23
23
27
13
75
17
15
13
17
17
33
25
23
27
27
17
13
25
19
21
7
19
23

* 又到了计算余弦相似度的时刻！

In [198]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [199]:
cosine_sim.shape

(9219, 9219)

In [200]:
cosine_sim

array([[1.        , 0.02441931, 0.02738955, ..., 0.        , 0.        ,
        0.        ],
       [0.02441931, 1.        , 0.        , ..., 0.02973505, 0.02500782,
        0.        ],
       [0.02738955, 0.        , 1.        , ..., 0.03335187, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02973505, 0.03335187, ..., 1.        , 0.08700222,
        0.        ],
       [0.        , 0.02500782, 0.        , ..., 0.08700222, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [201]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [202]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

In [203]:
get_recommendations('Mean Girls').head(10)

3319               Head Over Heels
4763                 Freaky Friday
1329              The House of Yes
6277              Just Like Heaven
7905         Mr. Popper's Penguins
7332    Ghosts of Girlfriends Past
6959     The Spiderwick Chronicles
8883                      The DUFF
6698         It's a Boy Girl Thing
7377       I Love You, Beth Cooper
Name: title, dtype: object

In [204]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]   
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [205]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
1134,Batman Returns,1706,6,1992,5.846862
132,Batman Forever,1529,5,1995,5.054144
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943
1260,Batman & Robin,1447,4,1997,4.287233


In [206]:
improved_recommendations('Mean Girls')

,title,vote_count,vote_average,year,wr
1547,The Breakfast Club,2189,7,1985,6.709602
390,Dazed and Confused,588,7,1993,6.254682
8883,The DUFF,1372,6,2015,5.818541
3712,The Princess Diaries,1063,6,2001,5.781086
4763,Freaky Friday,919,6,2003,5.757786
6277,Just Like Heaven,595,6,2005,5.681521
6959,The Spiderwick Chronicles,593,6,2008,5.680901
7494,American Pie Presents: The Book of Love,454,5,2009,5.119690
7332,Ghosts of Girlfriends Past,716,5,2009,5.092422
7905,Mr. Popper's Penguins,775,5,2011,5.087912


In [207]:
reader = Reader()
type(Reader())

In [208]:
ratings = pd.read_csv('/Users/apple/Desktop/5003-project/moviedata/TMD/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [209]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

In [210]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.8957
MAE:  0.6899
------------
Fold 2
RMSE: 0.8932
MAE:  0.6867
------------
Fold 3
RMSE: 0.8985
MAE:  0.6922
------------
Fold 4
RMSE: 0.8987
MAE:  0.6907
------------
Fold 5
RMSE: 0.8913
MAE:  0.6870
------------
------------
Mean RMSE: 0.8955
Mean MAE : 0.6893
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.6899446080839513,
                             0.6866640353512646,
                             0.692160981162863,
                             0.6906925062974029,
                             0.6870486898499264],
                            'rmse': [0.895741655975518,
                             0.8932048399514559,
                             0.8985225721833654,
                             0.8987289026347526,
                             0.8913414588419547]})

In [211]:
trainset = data.build_full_trainset()
svd.train(trainset)

In [212]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [213]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.478459649821828, details={'was_impossible': False})

In [214]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [215]:
id_map = pd.read_csv('/Users/apple/Desktop/5003-project/moviedata/TMD/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [217]:
indices_map = id_map.set_index('id')

In [218]:
# 对一个用户根据某个电影的评分，选取相似用户，然后取相似用户的电影评分作为结果。
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    print(idx)
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [221]:
hybrid(1, 'Avatar')

7488


,title,vote_count,vote_average,year,id,est
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.022471
974,Aliens,3282.0,7.7,1986,679,2.812701
1011,The Terminator,4208.0,7.4,1984,218,2.802806
922,The Abyss,822.0,7.1,1989,2756,2.802540
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,2.783535
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,2.729340
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.718556
3060,Sinbad and the Eye of the Tiger,39.0,6.3,1977,11940,2.684927
1376,Titanic,7770.0,7.5,1997,597,2.675789
344,True Lies,1138.0,6.8,1994,36955,2.670143


In [222]:
hybrid(500, 'Avatar')

7488


,title,vote_count,vote_average,year,id,est
3060,Sinbad and the Eye of the Tiger,39.0,6.3,1977,11940,3.483990
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,3.347876
2014,Fantastic Planet,140.0,7.6,1973,16306,3.290578
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.258852
974,Aliens,3282.0,7.7,1986,679,3.193973
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.163411
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.072631
1011,The Terminator,4208.0,7.4,1984,218,3.061568
4017,Hawk the Slayer,13.0,4.5,1980,25628,3.049808
344,True Lies,1138.0,6.8,1994,36955,3.040441
